In [1]:
import os

In [2]:
%pwd

'/Users/harshbhatt/Desktop/Projects/walmart-sales-forecast-mlops/research'

In [3]:
os.chdir("../")
%pwd

'/Users/harshbhatt/Desktop/Projects/walmart-sales-forecast-mlops'

In [4]:
import random
random.randint(1, 96)

6

In [18]:
from dataclasses import dataclass, field
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path 
    pipeline_name: str
    model_instance_name: str
    # alpha: float
    # l1_ratio: float
    target_column: str

In [7]:
from projectFiles.constants import *
from projectFiles.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        schema = self.schema
        # params = self.params

        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            pipeline_name = config.pipeline_name,
            model_instance_name = config.model_instance_name,
            # alpha = params.alpha,
            # l1_ratio = params.l1_ratio,
            target_column = schema.TARGET_COLUMN.name
        )

        return model_trainer_config

In [35]:
import numpy as np
import pandas as pd
import pickle
from lightgbm import LGBMRegressor
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from projectFiles import logger

In [36]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        target_col = self.config.target_column

        train_df = pd.read_csv(self.config.train_data_path)
        test_df = pd.read_csv(self.config.test_data_path)

        train_df.drop(columns=["Date"], inplace=True)
        test_df.drop(columns=["Date"], inplace = True)

        categorical_cols = [c for c in train_df.columns if train_df[c].dtype in [object]]
        numerical_cols = [c for c in train_df.columns if train_df[c].dtype in [float, int] and c != target_col]
        cycl_num_cols = [c for c in train_df.columns if ("sin" in str(c)) or ("cos" in str(c))]

        train_x = train_df.drop(columns=target_col)
        train_y = train_df[target_col]
        test_x = test_df.drop(columns=target_col)
        test_y = test_df[target_col]

        pipeline = make_pipeline(ColumnTransformer([("num", MinMaxScaler(), [c for c in numerical_cols if c not in cycl_num_cols])]),
                                 LGBMRegressor(n_jobs=-1, random_state=100))
        regressor = pipeline.fit(train_x, train_y)
        model_instance = regressor.named_steps["lgbmregressor"]

        pickle.dump(regressor, open(os.path.join(self.config.root_dir, self.config.pipeline_name), 'wb'))
        logger.info(f"Training Pipeline successfully saved at {self.config.root_dir}/{self.config.pipeline_name}")
        pickle.dump(model_instance, open(os.path.join(self.config.root_dir, self.config.model_instance_name), 'wb'))
        logger.info(f"Model Instance successfully saved at {self.config.root_dir}/{self.config.model_instance_name}")
    

In [37]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-03-02 21:54:32,762: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-02 21:54:32,764: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-02 21:54:32,766: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-02 21:54:32,767: INFO: common: created directory at: artifacts/model_trainer]
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005498 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2457
[LightGBM] [Info] Number of data points in the train set: 332778, number of used features: 14
[LightGBM] [Info] Start training from score 16015.343048
[2025-03-02 21:54:33,484: INFO: 1852907997: Training Pipeline successfully saved at artifacts/model_trainer/regressor_pipeline.pkl]
[2025-03-02 21:54:33,486: INFO: 1852907997: Model Instance successfully saved at artifacts/model_trainer/model.